<a href="https://colab.research.google.com/github/Suchit027/PCAP-Lab/blob/main/lab%205/add_q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
!chmod +x cuda_11.8.0_520.61.05_linux.run
!./cuda_11.8.0_520.61.05_linux.run --silent --toolkit

--2025-02-03 10:42:20--  https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.52.40.32, 23.52.40.19, 23.52.40.33, ...
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.52.40.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4336730777 (4.0G) [application/octet-stream]
Saving to: ‘cuda_11.8.0_520.61.05_linux.run’

cuda_11.8.0_520.61. 100%[===================>]   4.04G   191MB/s    in 31s     

2025-02-03 10:42:51 (135 MB/s) - ‘cuda_11.8.0_520.61.05_linux.run’ saved [4336730777/4336730777]



In [23]:
%%writefile add_q3.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void evenPhase(int *arr, int n, int phase) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int i = tid * 2;

    if (i < n - 1) {  // Ensure we don't go out of bounds
        if (arr[i] > arr[i + 1]) {
            int temp = arr[i];
            arr[i] = arr[i + 1];
            arr[i + 1] = temp;
        }
    }
}

__global__ void oddPhase(int *arr, int n, int phase) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int i = tid * 2 + 1;
    if (i < n - 1) {
        if (arr[i] > arr[i + 1]) {
            int temp = arr[i];
            arr[i] = arr[i + 1];
            arr[i + 1] = temp;
        }
    }
}

void oddEvenSort(int *arr, int n) {
    int *d_arr;
    cudaMalloc(&d_arr, n * sizeof(int));
    cudaMemcpy(d_arr, arr, n * sizeof(int), cudaMemcpyHostToDevice);

    int blockSize = 256;
    int gridSize = (n + blockSize - 1) / blockSize;

    for (int phase = 0; phase < n; phase++) {
        evenPhase<<<gridSize, blockSize>>>(d_arr, n, phase);
        cudaDeviceSynchronize();
        int temp[n];
        cudaMemcpy(temp, d_arr, n * sizeof(int), cudaMemcpyDeviceToHost);
        printf("After even phase %d: ", phase);
        for (int i = 0; i < n; i++) printf("%d ", temp[i]);
        printf("\n");
        oddPhase<<<gridSize, blockSize>>>(d_arr, n, phase);
        cudaDeviceSynchronize();
        cudaMemcpy(temp, d_arr, n * sizeof(int), cudaMemcpyDeviceToHost);
        printf("After odd phase %d: ", phase);
        for (int i = 0; i < n; i++) printf("%d ", temp[i]);
        printf("\n");
    }

    cudaMemcpy(arr, d_arr, n * sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(d_arr);
}

int main() {
    int n = 5;
    int arr[] = {5, 3, 1, 2, 4};

    printf("Initial array: ");
    for (int i = 0; i < n; i++) printf("%d ", arr[i]);
    printf("\n");

    oddEvenSort(arr, n);

    printf("Sorted array: ");
    for (int i = 0; i < n; i++) printf("%d ", arr[i]);
    printf("\n");

    return 0;
}




Overwriting add_q3.cu


In [24]:
!nvcc add_q3.cu -o add_q3.out

In [25]:
!./add_q3.out

Initial array: 5 3 1 2 4 
After even phase 0: 3 5 1 2 4 
After odd phase 0: 3 1 5 2 4 
After even phase 1: 1 3 2 5 4 
After odd phase 1: 1 2 3 4 5 
After even phase 2: 1 2 3 4 5 
After odd phase 2: 1 2 3 4 5 
After even phase 3: 1 2 3 4 5 
After odd phase 3: 1 2 3 4 5 
After even phase 4: 1 2 3 4 5 
After odd phase 4: 1 2 3 4 5 
Sorted array: 1 2 3 4 5 
